In [1]:
%autoreload 2

In [2]:
import torch,safetensors,pathlib
import huggingface_hub as hf_hub, safetensors as st
from nnsight import LanguageModel
from sae import Sae
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

Triton not installed, using eager implementation of SAE decoder.


In [3]:
def load_model(name:str,/,*,trust_remote_code:bool=False)->LanguageModel:
  model = LanguageModel(name,device_map="auto",
                        trust_remote_code=trust_remote_code,
                        low_cpu_mem_usage=True,torch_dtype=torch.float16)
  model.requires_grad_(False)
  return model

def load_manual_sae(name:str,hookpoint:str):
  repo_path = pathlib.Path(hf_hub.snapshot_download(name,allow_patterns=f"{hookpoint}/*"))
  if not ((hookpath:=repo_path/hookpoint)/'cfg.json').exists(): raise FileNotFoundError("require cfg.json to specify a specific layers")
  with st.safe_open(str(hookpath/'sae_weights.safetensors'),
                    framework='pt') as stf:
    return stf.get_tensor('W_dec')

In [10]:
prompt = "Logic is easy, but it is impossible to be logical to bitter end. It is considered truth if one"
feature = 6689
scale=20
max_new_tokens=40
num_return_sequences=10

In [5]:
sae = Sae.load_from_hub("EleutherAI/sae-llama-3.1-8b-32x", hookpoint="layers.29.mlp")

model = load_model("meta-llama/Meta-Llama-3.1-8B")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Dropping extra args {'signed': False}


In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [ ]:
with torch.inference_mode():
    outputs = model(**inputs, output_hidden_states=True)
    latent_acts = [sae.encode(outputs.hidden_states[29])]

In [ ]:
latent_acts

In [ ]:
l=latent_acts[0]
l.top_indices

In [9]:
sae.W_dec[[6689]] # feature index: logic

tensor([[ 0.0091, -0.0137, -0.0124,  ...,  0.0023,  0.0068, -0.0111]],
       grad_fn=<IndexBackward0>)

In [14]:
sae?

Signature:       sae(*args, **kwargs)
Type:            Sae
String form:    
Sae(
  (encoder): Linear(in_features=4096, out_features=131072, bias=True)
)
File:            ~/workspace/capstone/.venv/lib/python3.12/site-packages/sae/sae.py
Docstring:       <no docstring>
Class docstring:
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.

.. no